# Topic-Analyse durch Stichwort-Netzwerk

In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from collections import defaultdict

Ziel die Erstellung eines Graphen, der die Themenlandschaft von NTV widerspiegelt. Dafür verwendet werden die zuvor generierten Stichwörter. Die Knoten des Graphens von Stichwörtern, die häufiger vorkommen werden größer dargestellt als die von denen, die nicht häufig auftreten. Gleichzeitig sollen sich häufig auftretende Verbindungen anziehen und nicht häufig auftretende Verbindungen abstoßen.

Es werden zunächst die Daten eingelesen.

In [3]:
articles_data = pd.read_csv('2-4-data-extended-keywords-sentiment.csv')
articles = pd.DataFrame(articles_data)
articles.head()

,date,time,category,headline,text,upload,upload-hour,weekday,length,keywords,sentiment_headline,sentiment_text,sentiment_prob_headline_positive,sentiment_prob_headline_negative,sentiment_prob_headline_neutral,sentiment_prob_text_positive,sentiment_prob_text_negative,sentiment_prob_text_neutral
0,2024-05-10,17:14:00,politik,Gruppen treten am Vatertag mit Nazi Symbolik auf,Am Vatertag steuern Männer ein altes Militärfa...,2024-05-10 17:14:00,17,Friday,2908,"Politik, Gesellschaft, Extremismus, Rechtsextr...",neutral,neutral,0.000229,0.000604,0.999167,0.001950,0.010997,0.987053
1,2024-05-10,17:16:00,politik,Peking schimpft ueber US Zerstoerer im Suedchi...,China beansprucht die Paracel-Inseln im Südchi...,2024-05-10 17:16:00,17,Friday,3504,"China, USA, Paracel-Inseln, Südchinesisches Me...",negative,neutral,0.026734,0.750409,0.222857,0.002821,0.010704,0.986475
2,2024-05-10,19:33:00,politik,SPD Politiker Ecke schildert Sekunden der Prue...,Der vor einer Woche brutal attackierte SPD-Pol...,2024-05-10 19:33:00,19,Friday,2445,"Politik, Gewalt gegen Politiker, SPD, Wahlkamp...",negative,negative,0.005901,0.979033,0.015065,0.019161,0.757548,0.223291
3,2024-05-10,08:40:00,politik,Landkreise fordern haertere Buergergeld Sankti...,Beim Bürgergeld in seiner jetzigen Form bemäng...,2024-05-10 08:40:00,8,Friday,2984,"Bürgergeld, Landkreistag, Vermögensregelungen,...",neutral,neutral,0.000009,0.000026,0.999965,0.002032,0.010209,0.987759
4,2024-05-10,15:44:00,politik,Buschmanns Regierungsflieger kehrt in Parkposi...,Deutsche Regierungsflieger sind für ihre Panne...,2024-05-10 15:44:00,15,Friday,2570,"Regierungsflugzeug, Technikproblem, Verzögerun...",neutral,negative,0.000084,0.000253,0.999663,0.012173,0.830547,0.157280


Da die Keywords in einer Spalte stehen und nur durch Kommata getrennt sind, werden diese zunächst extrahiert, anhand der Kommata separiert und in einem neuen Dataframe gespeichert.

In [13]:
keywords_df = articles['keywords'].str.split(',', expand=True)
keywords_df = keywords_df.apply(lambda x: x.str.strip())
keywords_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,Politik,Gesellschaft,Extremismus,Rechtsextremismus,Polizei,Strafverfolgung,Sachsen,Nazisimbolik,Himmelfahrtstag,Volksverhetzung,None,None,None,None,None,None,None,None,None
1,China,USA,Paracel-Inseln,Südchinesisches Meer,Militär,Souveränität,Taiwan,Militärübung,Konflikt,Sicherheit,None,None,None,None,None,None,None,None,None
2,Politik,Gewalt gegen Politiker,SPD,Wahlkampf,Angriffe,gesellschaftliches Klima,Sachsen,Heilungsverlauf,AfD,Selbstjustiz,None,None,None,None,None,None,None,None,None
3,Bürgergeld,Landkreistag,Vermögensregelungen,Sanktionen,Arbeitsmarkt,CDU,Grundsicherung,Hartz IV,Reform,Existenzminimum,None,None,None,None,None,None,None,None,None
4,Regierungsflugzeug,Technikproblem,Verzögerung,G7,Justizminister,Künstliche Intelligenz,Pannen,Rückreise,Politische Reisen,Deutschland,None,None,None,None,None,None,None,None,None


In 12 Zeilen hat die Extrahierung nicht ordnungsgemäß funktioniert, was darauf zurückzuführen ist, dass die Keywords dort selbst Kommata enthalten und die Separierung nicht ordnungsgemäßg funktioniert hat. Dadurch besitzt das Dataframe mehr als nur 10 Spalten. Da die Anzahl an Fehlern - 12 - jedoch im Vergleich zum Gesamtdatensatz verschwindend gering ist, können diese Zeilen ignoriert werden.

In [11]:
invalid_rows = keywords_df[keywords_df[10].notna()]
invalid_rows

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
644,Samuel Koch,Querschnittslähmung,"""Wetten","dass..?""",Träume,Schauspiel,Kinderbuch,Theater,Ehefrau Sarah,Drehprojekte,Lebensperspektive,None,None,None,None,None,None,None,None
680,Politik,Thüringen,Sozialarbeit,Schulen,Haushaltsverhandlungen,Gesetz,Kinder- und Jugendhilfe,Opposition,Linke,SPD,Grüne,None,None,None,None,None,None,None,None
921,Politik,Bürgermeisterwahl,Oberhof,parteilos,Daniel Fischer,95,2 Prozent,Tourismus,Wintersport,Kommunalpolitik,Bevölkerung,None,None,None,None,None,None,None,None
1596,Hapag-Lloyd,Klimaneutralität,Atomenergie,Containerschiffe,Dekarbonisierung,Rolf Habben Jansen,Kosten,grüne Kraftstoffe,Transportvolumen,Politik,Wirtschaft,Gesellschaft,Umwelt,Energie,Schifffahrt,Technologie,Forschung,Nachhaltigkeit,Logistik
1867,Politik,Ukraine,NATO,Luftabwehr,Sicherheit,Truppen,Russland,Waffenhilfe,CDU,FDP,Grüne,None,None,None,None,None,None,None,None
3591,Rheinmetall,Bundeswehr,Großauftrag,Militär-Lastkraftwagen,3,5 Milliarden Euro,Sondervermögen,Rüstungsindustrie,Produktionserhöhung,Ukraine-Konflikt,Wirtschaft.,None,None,None,None,None,None,None,None
4548,Madonna,Welttournee,Konzert,Copacabana,1,5 Millionen,Brasilien,Gratis-Show,Pop-Musik,Feier,erfolgreich,None,None,None,None,None,None,None,None
4744,Fußball-Europameisterschaft,Snollebollekes,Party,niederländische Fans,Dortmund,Leipzig,Gastfreundschaft,gesellschaftliche Freude,Musik,Kultur,,None,None,None,None,None,None,None,None
4788,Brand,Holzbetrieb,Ostalb,Schaden,4,8 Millionen Euro,keine Verletzten,Feuerwehr,Einsatzkräfte,Ermittlungen,Ursachen unklar,None,None,None,None,None,None,None,None
6979,Daniel Brühl,Schauspieler,Regisseur,Karlovy Vary Filmfestival,Ehrenpreis,Präsidentenpreis,Becoming Karl Lagerfeld,Midlife-Crisis,Regiedebüt,Kinofilm,Oscars,None,None,None,None,None,None,None,None


Die überstehenden Spalten werden abgeschnitten.

In [14]:
keywords_df = keywords_df.iloc[:, :10]
keywords_df.head()

,0,1,2,3,4,5,6,7,8,9
0,Politik,Gesellschaft,Extremismus,Rechtsextremismus,Polizei,Strafverfolgung,Sachsen,Nazisimbolik,Himmelfahrtstag,Volksverhetzung
1,China,USA,Paracel-Inseln,Südchinesisches Meer,Militär,Souveränität,Taiwan,Militärübung,Konflikt,Sicherheit
2,Politik,Gewalt gegen Politiker,SPD,Wahlkampf,Angriffe,gesellschaftliches Klima,Sachsen,Heilungsverlauf,AfD,Selbstjustiz
3,Bürgergeld,Landkreistag,Vermögensregelungen,Sanktionen,Arbeitsmarkt,CDU,Grundsicherung,Hartz IV,Reform,Existenzminimum
4,Regierungsflugzeug,Technikproblem,Verzögerung,G7,Justizminister,Künstliche Intelligenz,Pannen,Rückreise,Politische Reisen,Deutschland


In [50]:
def create_graphml_file_for_gephi(filename: str):
    word_count = {}
    for index, row in keywords_df.iterrows():
        words = row.dropna().values
        for word in words:
            if word not in word_count:
                word_count[word] = 0
            word_count[word] += 1
    for index, row in keywords_df.iterrows():
        words = row.dropna().values
        for i, word1 in enumerate(words):
            if word_count[word1] > 10:
                for j in range(i + 1, len(words)):
                    word2 = words[j]
                    if word1 != word2 and word_count[word2] > 10:
                        edge = tuple(sorted([word1, word2]))
                        if not G.has_edge(word1, word2):
                            G.add_edge(word1, word2, weight=1)
                        else:
                            G.edges[edge]['weight'] += 1
    nodes_to_remove = [node for node in G.nodes if word_count[node] <= 10]
    G.remove_nodes_from(nodes_to_remove)
    nx.write_graphml(G, f"{filename}")

In [49]:
create_graphml_file_for_gephi("ntv-topic-graph.graphml")

Die so entstanden `graphml`-Datei kann anschließend in Gephi importiert werden.